# Baseline Implementation: BART

- Load data: `01_data_exploration`

- Data Preprocessing

- Baseline Model: BART

## Load Data

Copy from `01_data_exploration.ipynb`

- Github Setup

- Packages Setup

- Load Data

### Github Setup

In [2]:
username = 'shigenogoro'
repo_name = 'YouTube-Video-Summarization'
branch = 'kyle'

# Check if the destination directory already exists
import os
if os.path.exists(repo_name):
    print(f"The directory '{repo_name}' already exists. Skipping clone.")
    
    # Pull the latest changes
    %cd {repo_name}
    !git pull origin {branch}
else:
    # Clone the repository 
    !git clone https://github.com/{username}/{repo_name}.git -b {branch}
    
    # Change directory to the cloned repository
    %cd {repo_name}

Cloning into 'YouTube-Video-Summarization'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 91 (delta 28), reused 82 (delta 20), pack-reused 0 (from 0)
Receiving objects: 100% (91/91), 275.54 KiB | 21.19 MiB/s, done.
Resolving deltas: 100% (28/28), done.
/content/YouTube-Video-Summarization/YouTube-Video-Summarization


### Packages Setup

In [3]:
!pip install -r requirements.txt

In [4]:
# Download NLTK resources

!python -m spacy download en_core_web_sm

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 142.7 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

### Load Data

In [ ]:
from datasets import load_dataset

meetingbank = load_dataset("huuuyeah/meetingbank")

# Print the shape of the datasets
print(f"Train data shape: {meetingbank['train'].shape}")
print(f"Validation data shape: {meetingbank['validation'].shape}")
print(f"Test data shape: {meetingbank['test'].shape}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Train data shape: (5169, 4)
Validation data shape: (861, 4)
Test data shape: (862, 4)


In [7]:
import pandas as pd

# Split the dataset into train, validation, and test sets and convert to pandas DataFrames
train_df = pd.DataFrame(meetingbank['train'])
val_df = pd.DataFrame(meetingbank['validation'])
test_df = pd.DataFrame(meetingbank['test'])

## Data Preprocessing

In [14]:
# pull the latest changes
!git pull origin {branch}

remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 6 (delta 1), reused 6 (delta 1), pack-reused 0 (from 0)
Unpacking objects: 100% (6/6), 1.67 KiB | 1.67 MiB/s, done.
From https://github.com/shigenogoro/YouTube-Video-Summarization
 * branch            kyle       -> FETCH_HEAD
   c961341..08c9e3c  kyle       -> origin/kyle
Updating c961341..08c9e3c
Fast-forward
 src/__init__.py                      | 0
 {notebooks/src => src}/proprocess.py | 0
 2 files changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 src/__init__.py
 rename {notebooks/src => src}/proprocess.py (100%)


In [8]:
import torch

# Determine if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [15]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm.auto import tqdm
import os
from src.preprocess import preprocess_dataset

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')
print("BART tokenizer loaded.")

# Load the pre-trained BART model for summarization and move to device
model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-cnn').to(device)
print("BART model loaded and moved to device.")

# Define the path for the preprocessed data
preprocessed_data_path = 'data/meetingbank/preprocessed/'

# Preprocess the entire dataset and save to .tsv files
if not os.path.exists(preprocessed_data_path):
    os.makedirs(preprocessed_data_path, exist_ok=True)
    preprocess_dataset(train_df, os.path.join(preprocessed_data_path, os.path.join(preprocessed_data_path, 'train')))
    preprocess_dataset(val_df, os.path.join(preprocessed_data_path, os.path.join(preprocessed_data_path, 'validation')))
    preprocess_dataset(test_df, os.path.join(preprocessed_data_path, os.path.join(preprocessed_data_path, 'test')))
    print("Data preprocessing completed and saved.")

ModuleNotFoundError: No module named 'src.preprocess'

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm.auto import tqdm
import os
import torch

# --- Import Preprocessing API ---
# Make sure this path is correct relative to where your notebook is running
from src.preprocess import preprocess_dataset 
# --- End Import Preprocessing API ---

# Determine if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
print("BART tokenizer loaded.")

# Load the pre-trained BART model for summarization and move to device
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn").to(device)
print("BART model loaded and moved to device.")

# Define the path for the preprocessed test data
preprocessed_test_dir = "/content/drive/MyDrive/CS_685/Final_Project/data/preprocessed_test"
preprocessed_test_path = os.path.join(preprocessed_test_dir, "preprocessed_data_0.csv")

# Check if the preprocessed data file exists. If not, trigger preprocessing.
if not os.path.exists(preprocessed_test_path):
    print(f"Preprocessed test data not found at {preprocessed_test_path}. Starting preprocessing...")
    try:
        # Load the test split from the MeetingBank dataset
        meetingbank = load_dataset("huuuyeah/meetingbank")
        test_data = meetingbank['test']
        print("MeetingBank test data loaded for preprocessing.")
    except Exception as e:
        print(f"Error loading MeetingBank dataset: {e}. Please ensure datasets library is installed and you have internet access.")
        raise # Re-raise the exception if dataset loading fails

    # Call the external preprocessing function imported from preprocess.py
    preprocess_dataset(test_data, output_dir=preprocessed_test_dir)
    print("Preprocessing of test data complete.")
else:
    print(f"Preprocessed test data already exists at {preprocessed_test_path}.")

# Load the preprocessed test data sample
test_df_sample = pd.read_csv(preprocessed_test_path)
print(f"Loaded {len(test_df_sample)} samples from {preprocessed_test_path}")

# Tokenize the 'transcript' column and prepare inputs for the model
max_input_length = 1024
max_summary_length = 150
min_summary_length = 30
batch_size = 16 # Define batch size for batched inference

# Initialize a list to store results
bart_results = []

print("Generating summaries for the test data sample in batches...")

# Process in batches
for i in tqdm(range(0, len(test_df_sample), batch_size), desc="Generating BART summaries"):
    batch = test_df_sample.iloc[i : i + batch_size]

    batch_transcripts = batch['transcript'].tolist()
    batch_original_summaries = batch['summary'].tolist()
    batch_ids = batch['id'].tolist()

    # Tokenize the batch of transcripts
    inputs = tokenizer(
        batch_transcripts,
        max_length=max_input_length,
        truncation=True,
        padding="max_length", # Pad to max_input_length for batching
        return_tensors="pt"
    ).to(device) # Move inputs to the same device as the model

    # Generate summaries for the batch
    summary_ids = model.generate(
        inputs["input_ids"],
        num_beams=4,
        max_length=max_summary_length,
        min_length=min_summary_length,
        early_stopping=True
    )

    # Decode the generated summaries
    generated_summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    # Store results for each item in the batch
    for j in range(len(batch_ids)):
        bart_results.append({
            'id': batch_ids[j],
            'original_summary': batch_original_summaries[j],
            'generated_summary': generated_summaries[j]
        })

print("Summary generation complete.")

# Convert results to a DataFrame for easier handling and evaluation later
bart_baseline_df = pd.DataFrame(bart_results)
print(f"BART baseline results stored in DataFrame with {len(bart_baseline_df)} entries.")
print("First 5 BART generated summaries:")
print(bart_baseline_df[['id', 'original_summary', 'generated_summary']].head().to_string())